In [1]:
import medspacy

In [2]:
nlp = medspacy.load(enable=["sentencizer", "target_matcher", "context", "sectionizer"], load_rules=False)

In [3]:
import warnings
warnings.filterwarnings("ignore")

/Users/alecchapman/opt/anaconda3/envs/ssi/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
nlp.pipe_names

['sentencizer', 'target_matcher', 'context', 'sectionizer']

In [5]:
import pandas as pd

In [6]:
targets = pd.read_csv("./targets.tsv", sep="\t").fillna("")

In [7]:
modifiers = pd.read_csv("./modifiers.tsv", sep="\t").fillna("")

In [8]:
targets.head()

,Lex,Type,Unnamed: 2,Regex,Unnamed: 4,Direction,Codes
0,abnormal fluid,FLUID_COLLECTION,,,,,
1,abscess,FLUID_COLLECTION,(micro)*ab[a-z]*ess[es]*,,,,
2,abscesses,FLUID_COLLECTION,,,,,
3,air and fluid collection,FLUID_COLLECTION,air[ and/]*fluid collections*,,,,
4,air-fluid collection,FLUID_COLLECTION,air[-/ ]fluid collection,,,,


In [9]:
modifiers.head()

,Lex,Type,Regex,Direction,Unnamed: 4,Unnamed: 5,Unnamed: 6,Code
0,acute,ACUTE,,bidirectional,,,,
1,cannot be evaluated,AMBIVALENT_EXISTENCE,,bidirectional,,,,
2,incompletely evaluated,AMBIVALENT_EXISTENCE,,bidirectional,,,,
3,is in the differential,AMBIVALENT_EXISTENCE,is\sin\sthe\sdifferential,backward,,,,
4,is not excluded,AMBIVALENT_EXISTENCE,(is|was|are|were)\snot\sexcluded,backward,,,,


In [10]:
targets["Unnamed: 2"].value_counts()

                                                      21
loculations*                                           1
air[-/ ]fluid collection                               1
locules* of air|air locules*                           1
heterogeneous collections*                             1
(micro)*ab[a-z]*ess[es]*                               1
collections* of air|air collections*                   1
collections* of air[ and/]*fluid                       1
collections*                                           1
collections* of gas|gas collections*                   1
fluid complications*                                   1
fluid locules*                                         1
fluid[- ]{1}accumulation                               1
fluid[- ]re-*accumulation                              1
seromas*                                               1
fluid and air collections*                             1
(multi)*loculated[ a-z] (fluid)|(collections*)         1
non[- ]loculated fluid         

In [11]:
from medspacy.target_matcher import TargetRule

In [12]:
target_rules = []
for i, row in targets.iterrows():
    if row["Unnamed: 2"] == "":
        pattern = None
    else:
        pattern = row["Unnamed: 2"]
    target_rules.append(TargetRule(row["Lex"], row["Type"], pattern=pattern))

In [13]:
from medspacy.context import ConTextRule

In [14]:
context_rules = []
for i, row in modifiers.iterrows():
    if row["Regex"] == "":
        pattern = None
    else:
        pattern = row["Regex"]
    context_rules.append(ConTextRule(row["Lex"], row["Type"], pattern=pattern))

In [15]:
nlp.get_pipe("target_matcher").add(target_rules)

In [16]:
nlp.get_pipe("context").add(context_rules)

In [17]:
filepath = "./tutorial/sample_report.txt"

In [18]:
with open(filepath) as f:
    text = f.read()

In [19]:
doc = nlp(text)

In [20]:
from medspacy.visualization import visualize_ent

In [21]:
visualize_ent(doc)

In [22]:
target_matcher = nlp.get_pipe("target_matcher")

In [24]:
TargetRule.to_json(target_matcher.rules, "kb/targets.json")

In [25]:
context = nlp.get_pipe("context")

In [26]:
ConTextRule.to_json(context.rules, "kb/modifiers.json")